# Reprocessing procedure for Las Campanas Observatory Swope data using BANZAI-Imaging
## Matt Daily

## Set up relevant runtime configuration

In [1]:
import logging
from glob import glob
import os
import re

from banzai.calibrations import make_master_calibrations
from banzai import settings
from banzai import dbs
from banzai.utils.stage_utils import run_pipeline_stages
from banzai.logs import set_log_level
import banzai.main

import pkg_resources
import requests
from astropy.io import fits


set_log_level('DEBUG')
logger = logging.getLogger('banzai')

os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'
os.environ['DB_ADDRESS'] = 'sqlite:///test.db'

settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']
settings.no_bpm = True
settings.reduction_level = 91

# set up the context object.
context = banzai.main.parse_args(settings, parse_system_args=False)

ERROR:root:ConnectionError(MaxRetryError("HTTPConnectionPool(host='scheduler-dev.lco.gtn', port=80): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11dae39d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))


### Take the data from the directory and combine it into MEFs
This branch of BANZAI expects that each amplifier have an extension in an MEF. BANZAI can then do the work of stitching them together.

In [6]:

raw_data_dir = os.path.join(os.getcwd(), 'swope_data', 'raw')
mef_combined_data_dir = os.path.join(os.getcwd(), 'swope_data', 'mef_combined')

# TODO: Need to be able to parse filenames and split them into MEFs as necessary - this will need an outer loop.
prefix_set = set()
for file_path in glob(os.path.join(raw_data_dir, '*.fits.fz')):
    # extract the unique prefixes from the raw data directory
    prefix_set.add(re.sub('c\d', '', file_path))

# now create MEFs from the groupings!
for prefix in prefix_set:
    filepaths = [prefix.replace('.fits.fz', f'c{i}.fits.fz') for i in range(1,5)]

    image_hdus = fits.HDUList([fits.PrimaryHDU()])
    for path in filepaths:
        image_hdus.append(fits.open(path)['COMPRESSED_IMAGE'])

    # Now do some munging of the headers.
    # 1 is top right, rotate 90 counterclockwise and invert x (x -> -y, y-> -x)
    image_hdus[1].data = image_hdus[1].data.T[::-1, ::-1]
    image_hdus[1].header['DATASEC'] = '[129:2184,129:2176]'
    image_hdus[1].header['BIASSEC'] = '[1:128,1:2176]'
    image_hdus[1].header['DETSEC'] = '[2057:4112,2049:4096]'
    image_hdus[1].header.pop('TRIMSEC')

    # 2 is bottom left rotate 90 clockwise and invert x (y -> x x -> y)
    image_hdus[2].data = image_hdus[2].data.T
    image_hdus[2].header['DATASEC'] = '[1:2056,1:2048]'
    image_hdus[2].header['BIASSEC'] ='[2057:2084,1:2176]'
    image_hdus[2].header['DETSEC'] = '[1:2056,1:2048]'
    image_hdus[2].header.pop('TRIMSEC')

    # 3 bottom right rotate 90 counterclockwise (x-> -y, y - > x)
    image_hdus[3].data = image_hdus[3].data.T[:, ::-1]
    image_hdus[3].header['DATASEC'] = '[129:2184,1:2048]'
    image_hdus[3].header['BIASSEC'] ='[1:129,1:2176]'
    image_hdus[3].header['DETSEC'] = '[2057:4112,1:2048]'
    image_hdus[3].header.pop('TRIMSEC')

    # 4 is top left 270 deg  counterclockwise (x -> y, y->-x)
    image_hdus[4].data = image_hdus[4].data.T[::-1, :]
    image_hdus[4].header['DATASEC'] = '[1:2056,129:2176]'
    image_hdus[4].header['BIASSEC'] ='[2057:2184,1:2176]'
    image_hdus[4].header['DETSEC'] = '[1:2056, 2049:4096]'
    image_hdus[4].header.pop('TRIMSEC')

    # Copy all overlapping values into the main header
    for i in image_hdus[1].header:
        if i == 'COMMENT' or len(i) == 0:
            continue
        if image_hdus[2].header.get(i) == image_hdus[1].header.get(i):
            image_hdus[0].header[i] = image_hdus[1].header[i]

    image_hdus[0].header['TRIMSEC'] = '[1:4112,1:4096]'

    for i in range(1,5):
        image_hdus[i].header['RDNOISE'] = image_hdus[i].header['ENOISE']
        image_hdus[i].header['GAIN']  = image_hdus[i].header['EGAIN'] 

    # now create an HDUList and a unique filename to store the data in
    hdu_list = fits.HDUList(image_hdus)
    filename = f'{image_hdus[1].header["FILENAME"][:-2]}-{image_hdus[1].header["UT-DATE"].replace("-", "")}-combined.fits.fz'
    with open(os.path.join(mef_combined_data_dir, filename), 'wb') as f:
        hdu_list.writeto(f)

In [3]:
os.system(f'banzai_create_db --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_site --site LCO --latitude -29.08300 --longitude -70.698005 --elevation 2280 --timezone -4 --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site LCO --camera Direct/4Kx4K-4 --name Direct/4Kx4K-4 --instrument-type swope-imager --db-address={os.environ["DB_ADDRESS"]}')

0

In [4]:
from banzai.lco import LCOFrameFactory

frame_factory = LCOFrameFactory()
frame = frame_factory.open({'path': os.path.join(mef_combined_data_dir, filename) }, context)

In [5]:
[ccd.detector_section for ccd in frame.ccd_hdus]

### Now stitch the image

In [6]:
from banzai.mosaic import MosaicCreator

mosaic_stage = MosaicCreator(context)
image = mosaic_stage.do_stage(frame)

2023-03-20 10:15:45.195     INFO:          mosaic: Mosaicing image | {"filename": "ccd1144-20220925-combined.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "Object", "filter": "r", "processName": "MainProcess"}


INFO:banzai:Mosaicing image | {"filename": "ccd1144-20220925-combined.fits.fz", "site": "LCO", "instrument": "Direct/4Kx4K-4", "epoch": "20220925", "request_num": null, "obstype": "Object", "filter": "r", "processName": "MainProcess"}


In [8]:
intermediate_image = fits.HDUList([fits.PrimaryHDU(), fits.ImageHDU(image.data, image.meta)])
with open(os.path.join('/Users/mdaily/Documents/banzai/notebooks/swope_data/processed', filename.replace('combined', 'mosaicked-intermediate')), 'wb') as f:
    intermediate_image.writeto(f)